In [ ]:
import jax, flax, numpy
import flax.linen as nn
from notebookinit import *
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import optax
import tiktoken

2023-12-30 12:41:11.707657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 12:41:11.707689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 12:41:11.708963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 12:41:12.556659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Utilities blatantly missing from flax
def count_params(params):
    return sum(jax.tree_util.tree_leaves(jax.tree_util.tree_map(lambda x: x.size, params)))

# create the loss function
def make_loss(_model):
    @jax.jit
    def loss_fn(params, x_batched, y_batched):
      # Define the squared loss for a single pair (x,y)
      def squared_error(x, y):
        pred = _model.apply(params, x)
        return jnp.inner(y-pred, y-pred) / 2.0
      # Vectorize the previous to compute the average of the loss on all samples.
      return jnp.squeeze(jnp.mean(jax.vmap(squared_error)(x_batched,y_batched), axis=0))
    return loss_fn

In [ ]:
enc = tiktoken.encoding_for_model("gpt-4")

In [ ]:
enc.n_vocab

100277

In [ ]:
enc.encode("hi there")


[6151, 1070]

In [ ]:
indices_input = jnp.array(enc.encode("hi there"))
embedding_layer = nn.Embed(num_embeddings=enc.n_vocab, features=5)
embedding_vars = embedding_layer.init(jax.random.key(0), indices_input)
count_params(embedding_vars)

501385

In [ ]:
import numpy


In [ ]:
import lib_kata
text_encoded, text_decoder = lib_kata.load_dataset_and_tokenize()



NameError: name 'numpy' is not defined

In [ ]:
text_numpy.dtype

dtype('<U1')

In [ ]:
text_numpy.shape

(1115394,)

vocabulary: 
 ,e,t,o,a,i,h,s,r,n,
,l,d,u,m,y,w,,,c,f,g,b,p,:,k,v,.,',;,?,!,-,j,q,x,z,3,&,$ 
 size: 39 chars


In [ ]:
jnp.array([1,2,3], dtype=jnp.int32)

Array([1, 2, 3], dtype=int32)

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [ ]:
q_encoded = [stoi[x] for x in text_numpy]

In [ ]:
q_decoded = [itos[x] for x in q_encoded]
print(''.join(q_decoded[0:300]))

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you are all resolved rather to die than to famish?

all:
resolved. resolved.

first citizen:
first, you know caius marcius is chief enemy to the people.

all:
we know't, we know't.

first citizen:
let us


In [ ]:
vocabulary

array([' ', 'e', 't', 'o', 'a', 'i', 'h', 's', 'r', 'n', '\n', 'l', 'd',
       'u', 'm', 'y', 'w', ',', 'c', 'f', 'g', 'b', 'p', ':', 'k', 'v',
       '.', "'", ';', '?', '!', '-', 'j', 'q', 'x', 'z', '3', '&', '$'],
      dtype='<U1')

In [ ]:
numpy.argsort?

Signature:       numpy.argsort(a, axis=-1, kind=None, order=None)
Call signature:  numpy.argsort(*args, **kwargs)
Type:            _ArrayFunctionDispatcher
String form:     <function argsort>
File:            ~/prog/miniconda3/envs/py311/lib/python3.11/site-packages/numpy/core/fromnumeric.py
Docstring:      
Returns the indices that would sort an array.

Perform an indirect sort along the given axis using the algorithm specified
by the `kind` keyword. It returns an array of indices of the same shape as
`a` that index data along the given axis in sorted order.

Parameters
----------
a : array_like
    Array to sort.
axis : int or None, optional
    Axis along which to sort.  The default is -1 (the last axis). If None,
    the flattened array is used.
kind : {'quicksort', 'mergesort', 'heapsort', 'stable'}, optional
    Sorting algorithm. The default is 'quicksort'. Note that both 'stable'
    and 'mergesort' use timsort under the covers and, in general, the
    actual implementation wil

In [ ]:
numpy.sort?

Signature:       numpy.sort(a, axis=-1, kind=None, order=None)
Call signature:  numpy.sort(*args, **kwargs)
Type:            _ArrayFunctionDispatcher
String form:     <function sort>
File:            ~/prog/miniconda3/envs/py311/lib/python3.11/site-packages/numpy/core/fromnumeric.py
Docstring:      
Return a sorted copy of an array.

Parameters
----------
a : array_like
    Array to be sorted.
axis : int or None, optional
    Axis along which to sort. If None, the array is flattened before
    sorting. The default is -1, which sorts along the last axis.
kind : {'quicksort', 'mergesort', 'heapsort', 'stable'}, optional
    Sorting algorithm. The default is 'quicksort'. Note that both 'stable'
    and 'mergesort' use timsort or radix sort under the covers and, in general,
    the actual implementation will vary with data type. The 'mergesort' option
    is retained for backwards compatibility.

    .. versionchanged:: 1.15.0.
       The 'stable' option was added.

order : str or list of 

In [ ]:
shakespeare_encoded.shape

(301829,)

In [ ]:
jnp.unique(shakespeare_encoded).shape

(12111,)

In [ ]:
# observation: for a GPT-4 encoding, even a "reduced set" is still 12'111 unique tokens.
# for this toy model, it may be better to use unique lowercase characters only..

In [ ]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [ ]:
nn.activation.softmax(jnp.array([0.1, 0.2, 0.4]))

Array([0.28943312, 0.31987306, 0.39069384], dtype=float32)

In [ ]:
nn.activation.one_hot([0,1,2],3)

Array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)